# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [2]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

image_h = 8
image_w = 8
image_size = image_h * image_w
print_every = 100


device = '/CPU:0'

2024-05-04 10:26:15.855571: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-04 10:26:15.858375: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-04 10:26:15.889542: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 10:26:16.653994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [3]:
def load_dataset(test_proportion=0.2, val_proportion=0.2, flatten=True):
    digits = load_digits()
    digits_data = digits.data if flatten else digits.data.reshape(digits.data.shape[0], 8, 8, 1)
    X_train, X_val, y_train, y_val = train_test_split(digits_data, digits.target, test_size=val_proportion, random_state=1337)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_proportion / (1 - val_proportion), random_state=1337)

    mean_image = np.mean(X_train, axis=0)

    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(flatten=False)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (1077, 8, 8, 1)
Train labels shape:  (1077,) int64
Validation data shape:  (360, 8, 8, 1)
Validation labels shape:  (360,)
Test data shape:  (360, 8, 8, 1)
Test labels shape:  (360,)


In [4]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [5]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 8, 8, 1) (64,)
1 (64, 8, 8, 1) (64,)
2 (64, 8, 8, 1) (64,)
3 (64, 8, 8, 1) (64,)
4 (64, 8, 8, 1) (64,)
5 (64, 8, 8, 1) (64,)
6 (64, 8, 8, 1) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [6]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [7]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), activation='relu', padding='same',
                                            kernel_initializer=initializer)
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), activation='relu', padding='same',
                                            kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(128, activation='relu',
                                         kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                         kernel_initializer=initializer)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        scores = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [8]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 1, 8, 8))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [9]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False, print_every = 1, loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            # train_loss.reset_states()
            # train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        # val_loss.reset_states()
                        # val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1
        return train_accuracy.result()*100

In [10]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 8.210417747497559, Accuracy: 9.375, Val Loss: 10.521446228027344, Val Accuracy: 48.33333206176758
Iteration 1, Epoch 1, Loss: 10.623038291931152, Accuracy: 25.78125, Val Loss: 12.181952476501465, Val Accuracy: 46.9444465637207
Iteration 2, Epoch 1, Loss: 14.460156440734863, Accuracy: 30.20833396911621, Val Loss: 11.834972381591797, Val Accuracy: 50.83333206176758
Iteration 3, Epoch 1, Loss: 13.214972496032715, Accuracy: 37.5, Val Loss: 10.65766429901123, Val Accuracy: 53.95833206176758
Iteration 4, Epoch 1, Loss: 11.770947456359863, Accuracy: 44.375, Val Loss: 9.552871704101562, Val Accuracy: 58.999996185302734
Iteration 5, Epoch 1, Loss: 10.2921724319458, Accuracy: 51.5625, Val Loss: 8.214164733886719, Val Accuracy: 63.42593002319336
Iteration 6, Epoch 1, Loss: 8.943116188049316, Accuracy: 57.36607360839844, Val Loss: 7.321422100067139, Val Accuracy: 66.38888549804688
Iteration 7, Epoch 1, Loss: 8.249044418334961, Accuracy: 60.546875, Val Loss: 6.5761542320

<tf.Tensor: shape=(), dtype=float32, numpy=74.93037>

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [11]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn, print_every=1)

Iteration 0, Epoch 1, Loss: 8.384820938110352, Accuracy: 9.375, Val Loss: 4.125322341918945, Val Accuracy: 18.88888931274414
Iteration 1, Epoch 1, Loss: 6.278747081756592, Accuracy: 14.0625, Val Loss: 3.5103347301483154, Val Accuracy: 30.416667938232422
Iteration 2, Epoch 1, Loss: 5.4712371826171875, Accuracy: 19.791667938232422, Val Loss: 2.8384788036346436, Val Accuracy: 39.074073791503906
Iteration 3, Epoch 1, Loss: 4.416618824005127, Accuracy: 31.640625, Val Loss: 2.390758752822876, Val Accuracy: 46.59722137451172
Iteration 4, Epoch 1, Loss: 3.6945433616638184, Accuracy: 40.0, Val Loss: 2.076441764831543, Val Accuracy: 52.611114501953125
Iteration 5, Epoch 1, Loss: 3.190753936767578, Accuracy: 46.875, Val Loss: 1.8386211395263672, Val Accuracy: 57.268516540527344
Iteration 6, Epoch 1, Loss: 2.8249855041503906, Accuracy: 52.00892639160156, Val Loss: 1.6564840078353882, Val Accuracy: 61.07142639160156
Iteration 7, Epoch 1, Loss: 2.5706984996795654, Accuracy: 55.6640625, Val Loss: 1.5

<tf.Tensor: shape=(), dtype=float32, numpy=73.07335>

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [12]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (8, 8, 1)
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 8.17503833770752, Accuracy: 7.8125, Val Loss: 27.46299171447754, Val Accuracy: 30.55555534362793
Iteration 1, Epoch 1, Loss: 18.713043212890625, Accuracy: 14.84375, Val Loss: 28.305051803588867, Val Accuracy: 36.52777862548828
Iteration 2, Epoch 1, Loss: 22.9022274017334, Accuracy: 22.916667938232422, Val Loss: 26.341218948364258, Val Accuracy: 41.38888931274414


/home/slava/Documents/У(ч)ёба/DeepDarkLearningLabs/venv/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Iteration 3, Epoch 1, Loss: 20.746479034423828, Accuracy: 34.375, Val Loss: 22.761932373046875, Val Accuracy: 46.38888931274414
Iteration 4, Epoch 1, Loss: 18.26120948791504, Accuracy: 41.875, Val Loss: 19.05756378173828, Val Accuracy: 52.22222137451172
Iteration 5, Epoch 1, Loss: 15.977249145507812, Accuracy: 47.916664123535156, Val Loss: 16.331275939941406, Val Accuracy: 57.22222137451172
Iteration 6, Epoch 1, Loss: 14.332261085510254, Accuracy: 52.67857360839844, Val Loss: 14.332924842834473, Val Accuracy: 61.150794982910156
Iteration 7, Epoch 1, Loss: 12.831530570983887, Accuracy: 56.25, Val Loss: 12.775931358337402, Val Accuracy: 64.16666412353516
Iteration 8, Epoch 1, Loss: 11.788829803466797, Accuracy: 58.15972137451172, Val Loss: 11.636777877807617, Val Accuracy: 66.14197540283203
Iteration 9, Epoch 1, Loss: 11.01835823059082, Accuracy: 61.093746185302734, Val Loss: 10.599151611328125, Val Accuracy: 68.41666412353516
Iteration 10, Epoch 1, Loss: 10.109748840332031, Accuracy: 63

<tf.Tensor: shape=(), dtype=float32, numpy=72.88765>

Альтернативный менее гибкий способ обучения:

In [13]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9.1740 - sparse_categorical_accuracy: 0.5530 - val_loss: 0.4411 - val_sparse_categorical_accuracy: 0.9611
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.6551 - sparse_categorical_accuracy: 0.9240


[0.6617735624313354, 0.9166666865348816]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [14]:
learning_rate = 5e-4

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    input_shape = (8, 8, 1)
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Conv2D(channel_1, (5, 5), activation='relu', padding='same',
                               kernel_initializer=initializer, input_shape=input_shape),
        tf.keras.layers.Conv2D(channel_2, (3, 3), activation='relu', padding='same',
                               kernel_initializer=initializer),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)
    ]
    model = tf.keras.Sequential(layers)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################

    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################

    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 8.37335205078125, Accuracy: 18.75, Val Loss: 6.525502681732178, Val Accuracy: 15.833333015441895
Iteration 1, Epoch 1, Loss: 7.67636775970459, Accuracy: 14.84375, Val Loss: 5.696007251739502, Val Accuracy: 18.19444465637207
Iteration 2, Epoch 1, Loss: 6.644775390625, Accuracy: 15.625, Val Loss: 5.069661617279053, Val Accuracy: 19.629629135131836


/home/slava/Documents/У(ч)ёба/DeepDarkLearningLabs/venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Iteration 3, Epoch 1, Loss: 6.03685188293457, Accuracy: 17.1875, Val Loss: 4.601292610168457, Val Accuracy: 21.18055534362793
Iteration 4, Epoch 1, Loss: 5.4445600509643555, Accuracy: 19.375, Val Loss: 4.248658180236816, Val Accuracy: 23.05555534362793
Iteration 5, Epoch 1, Loss: 5.0841193199157715, Accuracy: 21.614582061767578, Val Loss: 3.9694344997406006, Val Accuracy: 24.953702926635742
Iteration 6, Epoch 1, Loss: 4.689236164093018, Accuracy: 23.660715103149414, Val Loss: 3.735741138458252, Val Accuracy: 26.944442749023438
Iteration 7, Epoch 1, Loss: 4.384458541870117, Accuracy: 25.9765625, Val Loss: 3.536801338195801, Val Accuracy: 28.71527671813965
Iteration 8, Epoch 1, Loss: 4.127628803253174, Accuracy: 27.95138931274414, Val Loss: 3.3539438247680664, Val Accuracy: 30.617284774780273
Iteration 9, Epoch 1, Loss: 3.919463634490967, Accuracy: 30.46875, Val Loss: 3.184562921524048, Val Accuracy: 32.58333206176758
Iteration 10, Epoch 1, Loss: 3.7008156776428223, Accuracy: 32.8125, Va

<tf.Tensor: shape=(), dtype=float32, numpy=43.73259>

In [15]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.6971 - sparse_categorical_accuracy: 0.4576 - val_loss: 0.4164 - val_sparse_categorical_accuracy: 0.8778
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4329 - sparse_categorical_accuracy: 0.8768 


[0.42036008834838867, 0.8805555701255798]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [16]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [17]:
input_shape = (8, 8, 1)

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 7.108091354370117, Accuracy: 15.625, Val Loss: 6.995588302612305, Val Accuracy: 18.61111068725586
Iteration 1, Epoch 1, Loss: 7.252679824829102, Accuracy: 15.625, Val Loss: 6.330205917358398, Val Accuracy: 19.86111068725586
Iteration 2, Epoch 1, Loss: 6.155717372894287, Accuracy: 19.791667938232422, Val Loss: 5.886967182159424, Val Accuracy: 22.592592239379883
Iteration 3, Epoch 1, Loss: 5.911129951477051, Accuracy: 19.53125, Val Loss: 5.504873752593994, Val Accuracy: 26.11111068725586
Iteration 4, Epoch 1, Loss: 5.638946056365967, Accuracy: 22.1875, Val Loss: 5.153144836425781, Val Accuracy: 28.44444465637207
Iteration 5, Epoch 1, Loss: 5.368155002593994, Accuracy: 25.52083396911621, Val Loss: 4.836570739746094, Val Accuracy: 30.509260177612305
Iteration 6, Epoch 1, Loss: 5.039126873016357, Accuracy: 28.34821319580078, Val Loss: 4.559291839599609, Val Accuracy: 33.015872955322266
Iteration 7, Epoch 1, Loss: 4.763949394226074, Accuracy: 29.6875, Val Loss: 4.

<tf.Tensor: shape=(), dtype=float32, numpy=44.382545>

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [20]:
num_epochs = 10

class CustomConvNet(tf.keras.Model):
    def __init__(self, num_filters, kernel_size, dense_units, dropout_rate, regularizer):
        super(CustomConvNet, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        self.conv1 = tf.keras.layers.Conv2D(num_filters, kernel_size, activation='relu', padding='same', kernel_initializer=initializer, kernel_regularizer=regularizer)
        self.conv2 = tf.keras.layers.Conv2D(num_filters, kernel_size, activation='relu', padding='same', kernel_initializer=initializer, kernel_regularizer=regularizer)
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer=initializer, kernel_regularizer=regularizer)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.fc2 = tf.keras.layers.Dense(10, activation='softmax', kernel_initializer=initializer)

    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=training)
        x = self.fc2(x)
        return x

# Перебор гиперпараметров
num_filters_list = [32, 64]
kernel_size_list = [(3, 3), (5, 5)]
dense_units_list = [128, 256]
dropout_rate_list = [0.3, 0.5]
regularizer_list = [None, tf.keras.regularizers.l2(0.001)]
loss_list = [
    tf.keras.losses.SparseCategoricalCrossentropy(),
]
optimizer_list = ['adam', 'sgd']
learning_rate_list = [1e-3, 1e-4]

best_accuracy = 0.0
best_params = None
results = []

for num_filters in num_filters_list:
    for kernel_size in kernel_size_list:
        for dense_units in dense_units_list:
            for dropout_rate in dropout_rate_list:
                for regularizer in regularizer_list:
                    for loss in loss_list:
                        for optimizer in optimizer_list:
                            for learning_rate in learning_rate_list:
                                def model_init_fn():
                                    return CustomConvNet(num_filters, kernel_size, dense_units, dropout_rate, regularizer)
    
                                def optimizer_init_fn():
                                    if optimizer == 'adam':
                                        return tf.keras.optimizers.Adam(learning_rate)
                                    elif optimizer == 'sgd':
                                        return tf.keras.optimizers.SGD(learning_rate)
                                    elif optimizer == 'rmsprop':
                                        return tf.keras.optimizers.RMSprop(learning_rate)
    
                                test_acc = train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True, print_every=160, loss_fn=loss)

                                current_params = {'num_filters': num_filters, 'kernel_size': kernel_size, 'dense_units': dense_units, 'dropout_rate': dropout_rate, 'regularizer': regularizer, 'loss_fn': loss.name, 'optimizer': optimizer, 'learning_rate': learning_rate, 'test_acc': test_acc}

                                results.append(current_params)
    
                                if test_acc > best_accuracy:
                                    best_accuracy = test_acc
                                    best_params = current_params


print(f"\nBest accuracy: {best_accuracy:.4f}")
print(f"Best parameters:", best_params)

Iteration 0, Epoch 1, Loss: 8.459726333618164, Accuracy: 7.8125, Val Loss: 3.5549399852752686, Val Accuracy: 33.61111068725586
Iteration 160, Epoch 10, Loss: 0.23766463994979858, Accuracy: 95.1004409790039, Val Loss: 1.8089393377304077, Val Accuracy: 65.69444274902344
Iteration 0, Epoch 1, Loss: 8.934349060058594, Accuracy: 17.1875, Val Loss: 7.315748691558838, Val Accuracy: 10.277777671813965
Iteration 160, Epoch 10, Loss: 1.0918951034545898, Accuracy: 76.34493255615234, Val Loss: 3.727236032485962, Val Accuracy: 52.63888931274414
Iteration 0, Epoch 1, Loss: 7.347914695739746, Accuracy: 12.5, Val Loss: 6.318758487701416, Val Accuracy: 20.0
Iteration 160, Epoch 10, Loss: 1.0310077667236328, Accuracy: 75.16902923583984, Val Loss: 3.304708480834961, Val Accuracy: 55.972225189208984
Iteration 0, Epoch 1, Loss: 11.715242385864258, Accuracy: 4.6875, Val Loss: 9.10726261138916, Val Accuracy: 10.277777671813965
Iteration 160, Epoch 10, Loss: 4.260796546936035, Accuracy: 28.897598266601562, Va

Опишите все эксперименты, результаты. Сделайте выводы.

In [ ]:
import pandas as pd

